<a href="https://colab.research.google.com/github/nmitchellward/ENV800_DLR/blob/main/SPP_Curtailment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENV 800 Final Project
Yale School of the Environment
May 2022
Noah Mitchell-Ward, Kyle Richmond-Crosset, Kirsten Williams, Sophia Ptacek

This code pulls generation mix and wind curtailment data from the Southwest Power Pool (SPP) and consolidates it for analysis.

In [61]:
import requests

import pandas as pd
import numpy as np

import io

from plotnine import *
import plotnine

import json

from google.colab import files

# Pulling curtailment data

In [2]:
# setting up curtailment dataframe
# data source: https://marketplace.spp.org/pages/ver-curtailments#%2F2021

curtailment = pd.DataFrame(columns = ['LocalIntervalEnding', 'GMTIntervalEnding', 'WindRedispatchCurtailments','WindManualCurtailments'])
curtailment

,LocalIntervalEnding,GMTIntervalEnding,WindRedispatchCurtailments,WindManualCurtailments


In [ ]:
#month array in string form for .csv urls
month = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [3]:
for i in month:

  # need to separate loops by number of days in the month 
  if i == '04' or i == '06' or i == '09' or i == '11':
    for j in range(1,31,1):
      day = str(j)
      if j < 10:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/ver-curtailments?path=%2F2021%2F' + i + '%2FVER-Curtailments-2021' + i + '0' + day + '.csv'
        curtailment = curtailment.append(pd.read_csv(csv_url))
      else:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/ver-curtailments?path=%2F2021%2F' + i + '%2FVER-Curtailments-2021' + i + day + '.csv'
        curtailment = curtailment.append(pd.read_csv(csv_url))
  elif i == '02':
    for j in range(1,29,1):
      day = str(j)
      if j < 10:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/ver-curtailments?path=%2F2021%2F' + i + '%2FVER-Curtailments-2021' + i + '0' + day + '.csv'
        curtailment = curtailment.append(pd.read_csv(csv_url))
      else:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/ver-curtailments?path=%2F2021%2F' + i + '%2FVER-Curtailments-2021' + i + day + '.csv'
        curtailment = curtailment.append(pd.read_csv(csv_url))
  else:  
    for j in range(1,32,1):
      day = str(j)
      if j < 10:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/ver-curtailments?path=%2F2021%2F' + i + '%2FVER-Curtailments-2021' + i + '0' + day + '.csv'
        curtailment = curtailment.append(pd.read_csv(csv_url))
      else:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/ver-curtailments?path=%2F2021%2F' + i + '%2FVER-Curtailments-2021' + i + day + '.csv'
        curtailment = curtailment.append(pd.read_csv(csv_url))

In [6]:
curtailment['TotalWindCurtailment'] = curtailment['WindRedispatchCurtailments'] + curtailment['WindManualCurtailments']

# Merging with generation data

In [7]:
# data source: https://marketplace.spp.org/pages/generation-mix-historical
generation = pd.read_csv('https://marketplace.spp.org/file-browser-api/download/generation-mix-historical?path=%2FGenMix_2021.csv')

In [8]:
generation.drop([0], inplace=True)

In [9]:
spp_df = pd.concat([generation.reset_index(drop=True), curtailment.reset_index(drop=True)], axis=1)
spp_df.shape

(105120, 27)

# Merging with marginal fuel data

In [37]:
# setting up marginal fuel dataframe
# data source: https://marketplace.spp.org/pages/fuel-on-margin#%2F2021

marginal_fuel = pd.DataFrame(columns = ['Interval', 'GMTIntervalEnd', 'Fuel On Margin'])

In [ ]:
for i in month:

  # need to separate loops by number of days in the month 
  if i == '04' or i == '06' or i == '09' or i == '11':
    for j in range(1,31,1):
      day = str(j)
      if j < 10:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + '0' + day + '0005.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
      else:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + day + '0005.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
  elif i == '02':
    for j in range(1,29,1):
      day = str(j)
      if j < 10:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + '0' + day + '0005.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
      elif j == 23:
        # first 5 min of this data frame is missing so it has a different URL
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + day + '0010.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
      else:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + day + '0005.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
  else:
    for j in range(1,32,1):
      day = str(j)
      if j < 10:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + '0' + day + '0005.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
      else:
        csv_url = 'https://marketplace.spp.org/file-browser-api/download/fuel-on-margin?path=%2F2021%2F' + i +'%2FFUEL-ON-MARGIN-2021' + i + day + '0005.csv'
        marginal_fuel = marginal_fuel.append(pd.read_csv(csv_url))
  

In [42]:
marginal_fuel.head()

,Interval,GMTIntervalEnd,Fuel On Margin
0,01/01/2021 00:05:00,01/01/2021 06:05:00,"GAS,COAL"
1,01/01/2021 00:10:00,01/01/2021 06:10:00,COAL
2,01/01/2021 00:15:00,01/01/2021 06:15:00,"WIND,GAS,COAL"
3,01/01/2021 00:20:00,01/01/2021 06:20:00,"WIND,GAS"
4,01/01/2021 00:25:00,01/01/2021 06:25:00,"GAS,COAL"


In [43]:
marginal_fuel['Date'] = pd.to_datetime(marginal_fuel['Interval'])
marginal_fuel.head()

,Interval,GMTIntervalEnd,Fuel On Margin,Date
0,01/01/2021 00:05:00,01/01/2021 06:05:00,"GAS,COAL",2021-01-01 00:05:00
1,01/01/2021 00:10:00,01/01/2021 06:10:00,COAL,2021-01-01 00:10:00
2,01/01/2021 00:15:00,01/01/2021 06:15:00,"WIND,GAS,COAL",2021-01-01 00:15:00
3,01/01/2021 00:20:00,01/01/2021 06:20:00,"WIND,GAS",2021-01-01 00:20:00
4,01/01/2021 00:25:00,01/01/2021 06:25:00,"GAS,COAL",2021-01-01 00:25:00


In [ ]:
spp_df['Date'] = pd.to_datetime(spp_df['LocalIntervalEnding'])

In [45]:
spp_df.head()

,LocalIntervalEnding,GMT MKT Interval,Coal Market,Coal Self,Diesel Fuel Oil Market,Diesel Fuel Oil Self,Hydro Market,Hydro Self,Natural Gas Market,Gas Self,...,Waste Heat Market,Waste Heat Self,Other Market,Other Self,Load,GMTIntervalEnding,WindRedispatchCurtailments,WindManualCurtailments,TotalWindCurtailment,Date
0,01/01/2021 00:05:00.000000,2021-01-01T06:05:00Z,6192.5,4226.2,0.0,0.0,34.7,286.1,3856.0,926.0,...,0.0,0.0,0.0,28.4,29527.813,01/01/2021 06:05:00.000000,71.32,0.0,71.32,2021-01-01 00:05:00
1,01/01/2021 00:10:00.000000,2021-01-01T06:10:00Z,6238.2,4232.6,0.0,0.0,33.9,285.1,3901.9,884.2,...,0.0,0.0,0.0,28.5,29463.581,01/01/2021 06:10:00.000000,74.41,0.0,74.41,2021-01-01 00:10:00
2,01/01/2021 00:15:00.000000,2021-01-01T06:15:00Z,6279.4,4265.1,0.0,0.0,33.5,283.5,3865.9,895.4,...,0.0,0.0,0.0,28.5,29364.982,01/01/2021 06:15:00.000000,106.44,0.0,106.44,2021-01-01 00:15:00
3,01/01/2021 00:20:00.000000,2021-01-01T06:20:00Z,6303.1,4249.4,0.0,0.0,33.6,285.9,3826.9,874.9,...,0.0,0.0,0.0,28.4,29367.888,01/01/2021 06:20:00.000000,111.06,0.0,111.06,2021-01-01 00:20:00
4,01/01/2021 00:25:00.000000,2021-01-01T06:25:00Z,6333.6,4263.7,0.0,0.0,33.0,283.3,3779.4,868.2,...,0.0,0.0,0.0,28.5,29352.684,01/01/2021 06:25:00.000000,112.79,0.0,112.79,2021-01-01 00:25:00


In [46]:
#merge dataframes on Date field
spp_df = spp_df.merge(marginal_fuel, how='left', on='Date')

In [48]:
spp_df.head()

,LocalIntervalEnding,GMT MKT Interval,Coal Market,Coal Self,Diesel Fuel Oil Market,Diesel Fuel Oil Self,Hydro Market,Hydro Self,Natural Gas Market,Gas Self,...,Other Self,Load,GMTIntervalEnding,WindRedispatchCurtailments,WindManualCurtailments,TotalWindCurtailment,Date,Interval,GMTIntervalEnd,Fuel On Margin
0,01/01/2021 00:05:00.000000,2021-01-01T06:05:00Z,6192.5,4226.2,0.0,0.0,34.7,286.1,3856.0,926.0,...,28.4,29527.813,01/01/2021 06:05:00.000000,71.32,0.0,71.32,2021-01-01 00:05:00,01/01/2021 00:05:00,01/01/2021 06:05:00,"GAS,COAL"
1,01/01/2021 00:10:00.000000,2021-01-01T06:10:00Z,6238.2,4232.6,0.0,0.0,33.9,285.1,3901.9,884.2,...,28.5,29463.581,01/01/2021 06:10:00.000000,74.41,0.0,74.41,2021-01-01 00:10:00,01/01/2021 00:10:00,01/01/2021 06:10:00,COAL
2,01/01/2021 00:15:00.000000,2021-01-01T06:15:00Z,6279.4,4265.1,0.0,0.0,33.5,283.5,3865.9,895.4,...,28.5,29364.982,01/01/2021 06:15:00.000000,106.44,0.0,106.44,2021-01-01 00:15:00,01/01/2021 00:15:00,01/01/2021 06:15:00,"WIND,GAS,COAL"
3,01/01/2021 00:20:00.000000,2021-01-01T06:20:00Z,6303.1,4249.4,0.0,0.0,33.6,285.9,3826.9,874.9,...,28.4,29367.888,01/01/2021 06:20:00.000000,111.06,0.0,111.06,2021-01-01 00:20:00,01/01/2021 00:20:00,01/01/2021 06:20:00,"WIND,GAS"
4,01/01/2021 00:25:00.000000,2021-01-01T06:25:00Z,6333.6,4263.7,0.0,0.0,33.0,283.3,3779.4,868.2,...,28.5,29352.684,01/01/2021 06:25:00.000000,112.79,0.0,112.79,2021-01-01 00:25:00,01/01/2021 00:25:00,01/01/2021 06:25:00,"GAS,COAL"


In [ ]:
# shift column 'Name' to first position
first_column = spp_df.pop('Date')
  
# insert column using insert(position,column_name,
# first_column) function
spp_df.insert(0, 'Date', first_column)

In [52]:
spp_df

,Date,LocalIntervalEnding,GMT MKT Interval,Coal Market,Coal Self,Diesel Fuel Oil Market,Diesel Fuel Oil Self,Hydro Market,Hydro Self,Natural Gas Market,...,Other Market,Other Self,Load,GMTIntervalEnding,WindRedispatchCurtailments,WindManualCurtailments,TotalWindCurtailment,Interval,GMTIntervalEnd,Fuel On Margin
0,2021-01-01 00:05:00,01/01/2021 00:05:00.000000,2021-01-01T06:05:00Z,6192.5,4226.2,0.0,0.0,34.7,286.1,3856.0,...,0.0,28.4,29527.813,01/01/2021 06:05:00.000000,71.32,0.00,71.32,01/01/2021 00:05:00,01/01/2021 06:05:00,"GAS,COAL"
1,2021-01-01 00:10:00,01/01/2021 00:10:00.000000,2021-01-01T06:10:00Z,6238.2,4232.6,0.0,0.0,33.9,285.1,3901.9,...,0.0,28.5,29463.581,01/01/2021 06:10:00.000000,74.41,0.00,74.41,01/01/2021 00:10:00,01/01/2021 06:10:00,COAL
2,2021-01-01 00:15:00,01/01/2021 00:15:00.000000,2021-01-01T06:15:00Z,6279.4,4265.1,0.0,0.0,33.5,283.5,3865.9,...,0.0,28.5,29364.982,01/01/2021 06:15:00.000000,106.44,0.00,106.44,01/01/2021 00:15:00,01/01/2021 06:15:00,"WIND,GAS,COAL"
3,2021-01-01 00:20:00,01/01/2021 00:20:00.000000,2021-01-01T06:20:00Z,6303.1,4249.4,0.0,0.0,33.6,285.9,3826.9,...,0.0,28.4,29367.888,01/01/2021 06:20:00.000000,111.06,0.00,111.06,01/01/2021 00:20:00,01/01/2021 06:20:00,"WIND,GAS"
4,2021-01-01 00:25:00,01/01/2021 00:25:00.000000,2021-01-01T06:25:00Z,6333.6,4263.7,0.0,0.0,33.0,283.3,3779.4,...,0.0,28.5,29352.684,01/01/2021 06:25:00.000000,112.79,0.00,112.79,01/01/2021 00:25:00,01/01/2021 06:25:00,"GAS,COAL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105139,2021-12-31 23:40:00,12/31/2021 23:40:00.000000,2022-01-01T05:40:00Z,2426.2,4388.3,0.0,0.0,54.7,363.6,1681.9,...,0.0,14.1,27734.387,01/01/2022 05:40:00.000000,1864.37,147.03,2011.40,12/31/2021 23:40:00,01/01/2022 05:40:00,"WIND,GAS"
105140,2021-12-31 23:45:00,12/31/2021 23:45:00.000000,2022-01-01T05:45:00Z,2412.7,4379.4,0.0,0.0,55.1,364.5,1678.7,...,0.0,14.4,27716.397,01/01/2022 05:45:00.000000,1945.04,199.47,2144.51,12/31/2021 23:45:00,01/01/2022 05:45:00,"WIND,GAS,COAL"
105141,2021-12-31 23:50:00,12/31/2021 23:50:00.000000,2022-01-01T05:50:00Z,2416.4,4381.6,0.0,0.0,54.7,364.2,1665.9,...,0.0,13.9,27688.465,01/01/2022 05:50:00.000000,2075.53,222.73,2298.26,12/31/2021 23:50:00,01/01/2022 05:50:00,"WIND,COAL"
105142,2021-12-31 23:55:00,12/31/2021 23:55:00.000000,2022-01-01T05:55:00Z,2399.8,4331.8,0.0,0.0,54.1,363.5,1660.7,...,0.0,14.3,27802.210,01/01/2022 05:55:00.000000,2374.31,219.41,2593.72,12/31/2021 23:55:00,01/01/2022 05:55:00,"WIND,GAS,COAL"


In [53]:
spp_df.drop(['GMT MKT Interval','GMTIntervalEnding','LocalIntervalEnding','Interval','GMTIntervalEnd'],axis=1,inplace=True)

In [54]:
spp_df

,Date,Coal Market,Coal Self,Diesel Fuel Oil Market,Diesel Fuel Oil Self,Hydro Market,Hydro Self,Natural Gas Market,Gas Self,Nuclear Market,...,Wind Self,Waste Heat Market,Waste Heat Self,Other Market,Other Self,Load,WindRedispatchCurtailments,WindManualCurtailments,TotalWindCurtailment,Fuel On Margin
0,2021-01-01 00:05:00,6192.5,4226.2,0.0,0.0,34.7,286.1,3856.0,926.0,0.0,...,13533.1,0.0,0.0,0.0,28.4,29527.813,71.32,0.00,71.32,"GAS,COAL"
1,2021-01-01 00:10:00,6238.2,4232.6,0.0,0.0,33.9,285.1,3901.9,884.2,0.0,...,13436.3,0.0,0.0,0.0,28.5,29463.581,74.41,0.00,74.41,COAL
2,2021-01-01 00:15:00,6279.4,4265.1,0.0,0.0,33.5,283.5,3865.9,895.4,0.0,...,13367.0,0.0,0.0,0.0,28.5,29364.982,106.44,0.00,106.44,"WIND,GAS,COAL"
3,2021-01-01 00:20:00,6303.1,4249.4,0.0,0.0,33.6,285.9,3826.9,874.9,0.0,...,13367.0,0.0,0.0,0.0,28.4,29367.888,111.06,0.00,111.06,"WIND,GAS"
4,2021-01-01 00:25:00,6333.6,4263.7,0.0,0.0,33.0,283.3,3779.4,868.2,0.0,...,13350.8,0.0,0.0,0.0,28.5,29352.684,112.79,0.00,112.79,"GAS,COAL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105139,2021-12-31 23:40:00,2426.2,4388.3,0.0,0.0,54.7,363.6,1681.9,708.4,0.0,...,17417.3,0.0,0.0,0.0,14.1,27734.387,1864.37,147.03,2011.40,"WIND,GAS"
105140,2021-12-31 23:45:00,2412.7,4379.4,0.0,0.0,55.1,364.5,1678.7,694.9,0.0,...,17356.9,0.0,0.0,0.0,14.4,27716.397,1945.04,199.47,2144.51,"WIND,GAS,COAL"
105141,2021-12-31 23:50:00,2416.4,4381.6,0.0,0.0,54.7,364.2,1665.9,708.3,0.0,...,17245.5,0.0,0.0,0.0,13.9,27688.465,2075.53,222.73,2298.26,"WIND,COAL"
105142,2021-12-31 23:55:00,2399.8,4331.8,0.0,0.0,54.1,363.5,1660.7,702.4,0.0,...,17215.5,0.0,0.0,0.0,14.3,27802.210,2374.31,219.41,2593.72,"WIND,GAS,COAL"


Removing rows with missing data

In [55]:
spp_df_clean = spp_df.dropna()

In [63]:
spp_df_clean.to_csv('spp_data_clean.csv') 
files.download('spp_data_clean.csv')

# spp_df_clean.to_csv('/Users/noahmitchellward/Downloads/spp_data_clean.csv', index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Plotting

In [ ]:
(ggplot(spp_df_clean)
    + geom_line(aes(x='Date', y='TotalWindCurtailment'), color='#009E73') 
    + scale_x_date(breaks = '12 months',date_labels = "%B")
    + xlab("Date")
    + ylab("Total Wind Curtailment (MWh)") 
    + ggtitle("Wind Curtailment in SPP, 2021") 
    + theme_classic() 
    + theme(axis_line=element_line(color="gray"), 
            axis_ticks=element_line(color = "gray"), 
            text=element_text(size = 12)) 
)